##### Copyright 2021 The TensorFlow Authors.

In [ ]:
# Apache License, 버전 2.0("라이선스")에 따라서만 라이선스가 허여되며, 
# 라이선스를 준수하는 경우에 한해 이 파일을 사용할 수 있습니다. 
# 라이선스 사본은 다음에서 받을 수 있습니다.
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# 이 코드는 묵시적 보증이나 소유권, 
# 특정 목적에의 적합성 또는 비침해성을 포함하되 그에 한정하지 않고
# 어떠한 종류의 명시적 또는 묵시적 보증도 없이 있는 그대로 제공됩니다.
# 라이선스에 따른 허가 및 제한에 적용되는 특정 언어는
# License를 참조하십시오. 

# TFX Pipeline 및 TensorFlow Model Analysis를 사용한 모델 분석


참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행"을 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfma"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

이 노트북 기반 튜토리얼에서는, 단순한 분류 모델을 만드는 TFX 파이프라인을 만들고 실행하여 여러 실행에서 그 성능을 분석합니다. 이 노트북은 [단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 빌드한 TFX 파이프라인을 기반으로 합니다. 아직 해당 튜토리얼을 읽지 않았다면, 이 노트북을 진행하기 전에 읽어보길 바랍니다.

모델을 수정하거나 이를 새로운 데이터세트로 훈련할 때, 모델을 개선했는지 혹은 모델이 악화되었는지 확인해야 합니다. 정확성과 같은 최상위 수준의 측정 메트릭만으로는 충분하지 않을 수 있습니다. 모든 훈련된 모델은 운영에 투입되기 전에 평가되어야 합니다.

이전 튜토리얼에서 만든 파이프라인에 `Evaluator` 구성 요소를 추가하겠습니다. Evaluator 구성 요소는 모델에 대한 심층 분석을 수행하고 새 모델을 기준과 비교하여 "충분히 우수"한지 결정합니다. 이는 [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma) 라이브러리를 사용하여 구현됩니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.

## 설정하기

설정 과정은 이전 튜토리얼과 동일합니다.

우선 TFX Python 패키지를 설치하고 모델에 사용할 데이터세트를 다운로드해야 합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인해야 합니다. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치


In [ ]:
!pip install -U tfx

### shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.

In [ ]:
!pip uninstall shapely -y

### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 런타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

TensorFlow 및 TFX 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 변수 설정하기

파이프라인을 정의하는 데 사용되는 변수가 몇 가지 있습니다. 이러한 변수를 원하는 대로 사용자 정의할 수 있습니다. 기본적으로 파이프라인의 모든 출력은 현재 디렉터리 아래에 생성됩니다.

In [ ]:
import os

PIPELINE_NAME = "penguin-tfma"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 예제 데이터 준비하기

동일한 [Palmer Penguins 데이터세트](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)를 사용하겠습니다.

이 데이터세트에는 이미 [0,1] 범위를 갖도록 정규화된 4개의 숫자 요소가 있습니다. 펭귄의 `species`를 예측하는 분류 모델을 구축할 것입니다.

TFX ExampleGen은 디렉터리로부터 입력을 읽기 때문에 디렉터리를 생성한 후 데이터세트를 디렉터리에 복사해야 합니다.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

## 파이프라인 생성하기

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 만든 파이프라인에 [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) 구성 요소를 추가하겠습니다.

Evaluator 구성 요소는 `ExampleGen` 구성 요소의 입력 데이터와 `Trainer` 구성 요소의 모델 및 [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) 객체가 필요합니다. 새로 훈련된 모델로 메트릭을 비교하는 데 사용할 수 있는 기준 모델을 선택적으로 제공할 수 있습니다.

Evaluator는 두 가지 종류의 출력 아티팩트인 `ModelEvaluation` 및 `ModelBlessing`를 생성합니다. ModelEvaluation은 TFMA 라이브러리를 통해 추가로 검사되거나 시각화될 수 있는 상세한 평가 결과를 포함합니다. ModelBlessing는 모델이 주어진 기준을 통과했는지에 대한 부울 결과를 포함하며 Pusher와 같은 후속 구성 요소에 신호로 사용될 수 있습니다.


### 모델 훈련 코드 작성하기

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서와 동일한 모델 코드를 사용합니다.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

### 파이프라인 정의 작성하기

함수를 정의하여 TFX 파이프라인을 만들겠습니다. 위에 언급된 Evaluator 구성 요소 외에, [`Resolver`](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver)라는 노드를 하나 더 추가하겠습니다. 이전 모델보다 새로운 모델이 더 나아지고 있는지 확인하기 위해 기준이라 불리는 이전 게시된 모델과 이를 비교해야 합니다. [ML Metadata(MLMD)](https://www.tensorflow.org/tfx/guide/mlmd)는 파이프라인의 모든 아티팩트를 추적하고 `Resolver`는  <code>LatestBlessedModelStrategy</code>라는 전략 클래스를 사용하여 MLMD의 가장 최근 <em>블레싱된</em> 모델을 찾을 수 있습니다.


In [ ]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # NEW: Get the latest blessed model for Evaluator.
  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  # NEW: Uses TFMA to compute evaluation statistics over features of a model and
  #   perform quality validation of a candidate model (compared to a baseline).

  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='species')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['species']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.6}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
  evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,

      # Following two components were added to the pipeline.
      model_resolver,
      evaluator,

      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

`eval_config`를 통해 Evaluator에 다음 정보를 제공해야 합니다.

- 구성할 추가 메트릭(모델에서 정의된 것보다 더 많은 메트릭을 원하는 경우).
- 구성할 슬라이스
- 검증이 포함된다면 검증할 모델 검증 임계값

`SparseCategoricalAccuracy`가 `model.compile()` 호출에 이미 포함되어 있었기 때문에 이는 분석에 자동으로 추가됩니다. 따라서 여기에서 추가 메트릭을 추가하지 않습니다. `SparseCategoricalAccuracy` 역시 모델이 충분히 우수한지 결정하는데 사용됩니다.

전체 데이터세트 및 각 펭귄 종에 대한 메트릭을 계산합니다. `SlicingSpec`은 선언된 메트릭을 집계하는 방법을 지정합니다.

새로운 모델이 통과해야 하는 두 개의 임계값이 있으며, 하나는 절대 임계값 0.6이며 다른 하나는 기준 모델보다 높아야 하는 상대 임계값입니다. 처음으로 파이프라인을 실행하는 경우 `change_threshold`는 무시되며 value_threshold만 확인됩니다. 두 번 이상 파이프라인을 실행하는 경우, `Resolver`는 이전 실행에서 모델을 찾고 비교를 위해 기준 모델로서 사용됩니다.

자세한 내용은 [Evaluator 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/evaluator#using_the_evaluator_component)를 참조하세요.

## 파이프라인 실행


이전 튜토리얼과 같이 `LocalDagRunner`를 사용합니다.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

파이프라인이 완료되면 다음과 같은 것이 표시되어야 합니다.

```
INFO:absl:Blessing result True written to pipelines/penguin-tfma/Evaluator/blessing/4.
```

또는 생성된 아티팩터가 저장된 출력 디렉터리를 수동으로 확인할 수도 있습니다. 파일 탐색기로 `pipelines/penguin-tfma/Evaluator/blessing/`으로 이동하는 경우 평가 결과에 따라 이름이 `BLESSED` 또는 `NOT_BLESSED`인 파일을 볼 수 있습니다.

블레싱 결과가 `False`라면, 모델이 운영에 사용하는데 충분히 우수하지 않기 때문에 Pusher는 모델을 `serving_model_dir`로 투입하길 거부합니다.

다른 평가 구성을 통해 아마도 다시 파이프라인을 실행할 수 있습니다. 완전히 동일한 구성과 데이터세트로 파이프라인을 실행한다 해도, 모델 훈련의 내재하는 무작위성으로 인해 훈련된 모델은 약간 다를 수 있으며. 이는 `NOT_BLESSED` 모델로 이어질 수 있습니다.

### 파이프라인의 출력 검사

TFMA를 사용하여 ModelEvaluation 아티팩트에서 평가 결과를 검사하고 시각화할 수 있습니다.

> **참고: Colab에 접속하지 않았다면, Jupyter 확장 프로그램을 설치하세요.** TFMA에서 시각화를 확인하려면 TensorFlow Model Analysis 확장 프로그램이 필요합니다. 이 확장 프로그램은 이미 Google Colab에 설치되어 있지만, 이 노트북을 다른 환경에서 실행하는 경우 설치해야 할 수 있습니다. [설치 가이드](https://github.com/tensorflow/model-analysis#installation)에서 Jupyter 확장 프로그램 설치 안내를 확인하세요.


#### 출력 아티팩트에서 분석 결과 받기

이러한 출력을 MLMD API를 사용하여 프로그래밍 방식으로 찾을 수 있습니다. 우선, 일부 유틸리티 함수를 정의하여 방금 생성한 출력 아티팩트를 찾을 수 있습니다.

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)


`Evaluator` 구성 요소의 최근 실행을 찾고 출력 아티팩트를 얻을 수 있습니다.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  evaluator_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                          'Evaluator')
  eval_artifact = evaluator_output[standard_component_specs.EVALUATION_KEY][0]

`Evaluator`는 항상 하나의 평가 아티팩트를 반환하며 TensorFlow Model Analysis 라이브러리를 사용하여 이를 시각화할 수 있습니다. 예를 들어, 다음 코드는 각 펭귄 종에 대한 정확성 메트릭을 렌더링합니다.

In [ ]:
import tensorflow_model_analysis as tfma

eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='species')

`Show` 드롭다운 목록에서 'sparse_categorical_accuracy'를 선택하면 종별 정확성 값을 볼 수 있습니다. 슬라이스를 몇 개 더 추가하고 모델이 모든 분포에 적합한지, 가능한 편향이 존재하는지 확인할 수 있습니다.

## 다음 단계

[TensorFlow Model Analysis 라이브러리 튜토리얼](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic)에서 모델 분석에 대해 더 자세히 알아보세요.

https://www.tensorflow.org/tfx/tutorials에서 더 많은 리소스를 확인할 수 있습니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.
